# Detecting Image Manipulation with VAAS (Quick Start)

Visual media is increasingly easy to manipulate, yet difficult to verify.
This notebook demonstrates how to use VAAS to assess whether an image
contains visual inconsistencies that may indicate manipulation.

The goal here is simple:
- load a pretrained VAAS model,
- run inference on a single image,
- and interpret the resulting scores.

No prior knowledge of VAAS internals is required.


## Installation

VAAS is distributed as a lightweight, inference-first Python library.
PyTorch is required at runtime for model execution.

In Colab, installing VAAS via pip is sufficient.


In [ ]:
!pip install vaas==0.1.7

In [ ]:
from vaas.inference.pipeline import VAASPipeline
from PIL import Image
import requests
from io import BytesIO

## Loading the Pretrained VAAS Model

VAAS models are distributed through Hugging Face.
Here we load a publicly available checkpoint trained for visual anomaly detection.

The `alpha` parameter controls how feature-level and patch-level inconsistencies
are combined. For a quick start, the default value of 0.5 is appropriate.


In [ ]:
pipeline = VAASPipeline.from_pretrained(
    "OBA-Research/vaas-v1-df2023",
    device="cpu",   # change to "cuda" if running on a GPU
    alpha=0.5,
)

## Loading an Example Image

For convenience and reproducibility, we load an image directly from the VAAS
GitHub repository. The image is fetched over HTTPS and processed in memory.


In [ ]:
image_url = (
    "https://raw.githubusercontent.com/OBA-Research/VAAS/main/"
    "examples/images/COCO_DF_C110B00000_00539519.jpg"
)

response = requests.get(image_url)
image = Image.open(BytesIO(response.content)).convert("RGB")

image


## Running Inference

VAAS returns three scores:

- S_F: feature-level inconsistency score
- S_P: patch-level inconsistency score
- S_H: hybrid score combining S_F and S_P

S_H is typically used as the primary indicator of visual inconsistency.


In [ ]:
result = pipeline(image)
print(f"Feature consistency score (S_F): {result['S_F']:.4f}")
print(f"Patch consistency score   (S_P): {result['S_P']:.4f}")
print(f"Hybrid anomaly score      (S_H): {result['S_H']:.4f}")
print("Anomaly map shape:", result["anomaly_map"].shape)

## Visualizing Suspicious Regions

VAAS can also generate a visualization highlighting regions that contribute
most strongly to the anomaly score.

In [ ]:
pipeline.visualize(
    image=image,
    save_path="vaas_visualization.png",
    mode="all",
    threshold=0.5,
)

In [ ]:
from IPython.display import Image as IPImage, display
display(IPImage("vaas_visualization.png"))

You can explore the next notebook on [02_where_was_the_image_manipulated.ipynb](https://colab.research.google.com/drive/1EBZYx56DQcTaxPlP_hWCnXaVDzjcv_TV?usp=sharing)